In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 200)


In [26]:
import pandas as pd

df = pd.read_excel("cleaned_merged.csv.xlsx")  # Because it's Excel format
df['date'] = pd.to_datetime(df['date'])  # Ensure date column is in datetime format

print(df.shape)
df.head()


(1080, 11)


,date,region,resource_type,usage_cpu,usage_storage,users_active,economic_index,cloud_market_demand,holiday,Day_of_Week,Month
0,2023-01-01,EAST US,VM,88,1959,470,104.97,0.99,1,6,1
1,2023-01-01,EAST US,Storage,92,1595,388,104.97,0.99,1,6,1
2,2023-01-01,EAST US,Container,70,621,414,104.97,0.99,1,6,1
3,2023-01-01,WEST US,VM,60,1982,287,104.97,0.99,1,6,1
4,2023-01-01,WEST US,Storage,85,1371,351,104.97,0.99,1,6,1


In [27]:
# Add time-based features
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['quarter'] = df['date'].dt.quarter


df.head()


,date,region,resource_type,usage_cpu,usage_storage,users_active,economic_index,cloud_market_demand,holiday,Day_of_Week,Month,month,day_of_week,is_weekend,quarter
0,2023-01-01,EAST US,VM,88,1959,470,104.97,0.99,1,6,1,1,6,1,1
1,2023-01-01,EAST US,Storage,92,1595,388,104.97,0.99,1,6,1,1,6,1,1
2,2023-01-01,EAST US,Container,70,621,414,104.97,0.99,1,6,1,1,6,1,1
3,2023-01-01,WEST US,VM,60,1982,287,104.97,0.99,1,6,1,1,6,1,1
4,2023-01-01,WEST US,Storage,85,1371,351,104.97,0.99,1,6,1,1,6,1,1


In [28]:
# Group by region and resource_type for time-series operations
grp = df.groupby(['region', 'resource_type'], group_keys=False)

# Add lag features for CPU usage
df['usage_cpu_lag1'] = grp['usage_cpu'].shift(1)
df['usage_cpu_lag3'] = grp['usage_cpu'].shift(3)
df['usage_cpu_lag7'] = grp['usage_cpu'].shift(7)
df.head()



,date,region,resource_type,usage_cpu,usage_storage,users_active,economic_index,cloud_market_demand,holiday,Day_of_Week,Month,month,day_of_week,is_weekend,quarter,usage_cpu_lag1,usage_cpu_lag3,usage_cpu_lag7
0,2023-01-01,EAST US,VM,88,1959,470,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN
1,2023-01-01,EAST US,Storage,92,1595,388,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN
2,2023-01-01,EAST US,Container,70,621,414,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN
3,2023-01-01,WEST US,VM,60,1982,287,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN
4,2023-01-01,WEST US,Storage,85,1371,351,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN


In [29]:
# Rolling mean, max, min for CPU usage
df['cpu_roll_mean_7'] = grp['usage_cpu'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
df['cpu_roll_max_7']  = grp['usage_cpu'].transform(lambda x: x.rolling(window=7, min_periods=1).max())
df['cpu_roll_min_7']  = grp['usage_cpu'].transform(lambda x: x.rolling(window=7, min_periods=1).min())

df['cpu_roll_mean_30'] = grp['usage_cpu'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
df['cpu_roll_max_30']  = grp['usage_cpu'].transform(lambda x: x.rolling(window=30, min_periods=1).max())
df['cpu_roll_min_30']  = grp['usage_cpu'].transform(lambda x: x.rolling(window=30, min_periods=1).min())
df.head()

,date,region,resource_type,usage_cpu,usage_storage,users_active,economic_index,cloud_market_demand,holiday,Day_of_Week,Month,month,day_of_week,is_weekend,quarter,usage_cpu_lag1,usage_cpu_lag3,usage_cpu_lag7,cpu_roll_mean_7,cpu_roll_max_7,cpu_roll_min_7,cpu_roll_mean_30,cpu_roll_max_30,cpu_roll_min_30
0,2023-01-01,EAST US,VM,88,1959,470,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,88.0,88.0,88.0,88.0,88.0,88.0
1,2023-01-01,EAST US,Storage,92,1595,388,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,92.0,92.0,92.0,92.0,92.0,92.0
2,2023-01-01,EAST US,Container,70,621,414,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,70.0,70.0,70.0,70.0,70.0,70.0
3,2023-01-01,WEST US,VM,60,1982,287,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,60.0,60.0,60.0,60.0,60.0,60.0
4,2023-01-01,WEST US,Storage,85,1371,351,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,85.0,85.0,85.0,85.0,85.0,85.0


In [30]:
# ================================
#  Derived Metrics 
# ================================

# CPU Utilization = cpu_used / cpu_total
# - cpu_total = maximum observed CPU usage for that resource_type
df['cpu_total'] = df.groupby('resource_type')['usage_cpu'].transform('max')
df['cpu_utilization'] = df['usage_cpu'] / df['cpu_total']

# Storage Efficiency = storage_used / storage_allocated
# - storage_allocated = maximum observed storage usage for that resource_type
df['storage_allocated'] = df.groupby('resource_type')['usage_storage'].transform('max')
df['storage_efficiency'] = df['usage_storage'] / df['storage_allocated']

df.head()


,date,region,resource_type,usage_cpu,usage_storage,users_active,economic_index,cloud_market_demand,holiday,Day_of_Week,Month,month,day_of_week,is_weekend,quarter,usage_cpu_lag1,usage_cpu_lag3,usage_cpu_lag7,cpu_roll_mean_7,cpu_roll_max_7,cpu_roll_min_7,cpu_roll_mean_30,cpu_roll_max_30,cpu_roll_min_30,cpu_total,cpu_utilization,storage_allocated,storage_efficiency
0,2023-01-01,EAST US,VM,88,1959,470,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,88.0,88.0,88.0,88.0,88.0,88.0,99,0.888889,1994,0.982447
1,2023-01-01,EAST US,Storage,92,1595,388,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,92.0,92.0,92.0,92.0,92.0,92.0,99,0.929293,1992,0.800703
2,2023-01-01,EAST US,Container,70,621,414,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,70.0,70.0,70.0,70.0,70.0,70.0,99,0.707071,1995,0.311278
3,2023-01-01,WEST US,VM,60,1982,287,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,60.0,60.0,60.0,60.0,60.0,60.0,99,0.606061,1994,0.993982
4,2023-01-01,WEST US,Storage,85,1371,351,104.97,0.99,1,6,1,1,6,1,1,NaN,NaN,NaN,85.0,85.0,85.0,85.0,85.0,85.0,99,0.858586,1992,0.688253


In [31]:
import numpy as np
np.random.seed(42)
df['weather_index'] = np.random.randint(0, 3, size=len(df))  # 0=normal, 1=hot, 2=cold
df['power_outage_flag'] = np.random.choice([0, 1], size=len(df), p=[0.98, 0.02])
df['price_change'] = np.random.uniform(-0.05, 0.05, size=len(df))  # ±5% change


In [32]:
# Fill missing values caused by lag and rolling operations
df.fillna(method='bfill', inplace=True)
df.fillna(method='ffill', inplace=True)

# Check if any NaNs are left
print("Remaining missing values:", df.isna().sum().sum())


Remaining missing values: 0


C:\Users\trisha\AppData\Local\Temp\ipykernel_24748\3832863174.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
C:\Users\trisha\AppData\Local\Temp\ipykernel_24748\3832863174.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [33]:
# One-hot encode 'region' and 'resource_type'
df = pd.get_dummies(df, columns=['region', 'resource_type'], drop_first=False)  # I recommend drop_first=False

print("Categorical variables one-hot encoded.")
df.head()

# ------------------------
# Add string columns for frontend filters
# ------------------------
df['Region'] = ''
df.loc[df['region_EAST US'] == 1, 'Region'] = 'East US'
df.loc[df['region_NORTH EUROPE'] == 1, 'Region'] = 'North Europe'
df.loc[df['region_SOUTHEAST ASIA'] == 1, 'Region'] = 'Southeast Asia'
df.loc[df['region_WEST US'] == 1, 'Region'] = 'West US'

df['ResourceType'] = ''
df.loc[df['resource_type_VM'] == 1, 'ResourceType'] = 'VM'
df.loc[df['resource_type_Storage'] == 1, 'ResourceType'] = 'Storage'
df.loc[df['resource_type_Container'] == 1, 'ResourceType'] = 'Container'


Categorical variables one-hot encoded.


In [34]:
df.to_csv("milestone2_feature_engineered.csv", index=False)
